In [137]:
# Source - https://stackoverflow.com/a/47022213
# Posted by Isma, modified by community. See post 'Timeline' for change history
# Retrieved 2026-02-23, License - CC BY-SA 4.0

import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
 

In [178]:
import asyncio
import pandas as pd
from azure.identity.aio import ClientSecretCredential
from msgraph import GraphServiceClient
import requests
from msal import ConfidentialClientApplication
from msgraph.generated.sites.item.lists.item.items.items_request_builder import ItemsRequestBuilder
from msgraph.generated.models.list_item import ListItem
from msgraph.generated.models.field_value_set import FieldValueSet


In [ ]:

#this chunk of code retrives the Site ID of the OPI sharepoint
authority = f"https://login.microsoftonline.com/bmore.onmicrosoft.com"
scopes = ["https://graph.microsoft.com/.default"]

TENANT_ID = 'bmore.onmicrosoft.com'
CLIENT_ID = ''
CLIENT_SECRET = ''

#connecting to microsoft app to get access token so requests.get can work
app = ConfidentialClientApplication(
    CLIENT_ID,
    authority=authority,
    client_credential=CLIENT_SECRET,
)

token_response = app.acquire_token_for_client(scopes=scopes)

if "access_token" not in token_response:
    raise Exception(token_response)

access_token = token_response['access_token']

headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json"
}

#retrieving the site id which will be needed to use the Graph API
site_url = "https://graph.microsoft.com/v1.0/sites/bmore.sharepoint.com:/sites/opi"

site_response = requests.get(site_url, headers=headers)
site_response.raise_for_status()

site_id = site_response.json()["id"]

In [ ]:
#retrives all list/list id pairs on the SP page
lists_url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/lists"

lists_response = requests.get(lists_url, headers=headers)
lists_response.raise_for_status()
lists = lists_response.json()["value"]

#retriveing the list ids for the specific lists that we are interested in
list_ids = []
list_titles = ['Book of Business - Initiatives','Book of Business - Reporting Periods','Book of Business - Weekly Updates']

for list in list_titles:
    target_list = next((l for l in lists if l["displayName"] == list), None)

    if not target_list:
        raise Exception(f"List '{list}' not found.")

    list_ids.append(target_list["id"])
    print(f"List Name: {list}, List ID: {target_list["id"]}")

credential = ClientSecretCredential(
    TENANT_ID,
    CLIENT_ID,
    CLIENT_SECRET
)

async def sp_list_to_df(site_id,list_id,credential):
    
    #connect to graph API client and prepare query parameters
    graph_client = GraphServiceClient(credential)

    query_params = ItemsRequestBuilder.ItemsRequestBuilderGetQueryParameters(
        expand=["fields"]
    )

    request_config = ItemsRequestBuilder.ItemsRequestBuilderGetRequestConfiguration(
        query_parameters=query_params
    )

    #using the site and list ID to retrieve the items from the SP list
    #also retrieving column names to help with reuploading the list back to SP
    result = await graph_client.sites.by_site_id(site_id).lists.by_list_id(list_id).items.get(request_configuration=request_config)
    columns = await graph_client.sites.by_site_id(site_id).lists.by_list_id(list_id).columns.get()
    column_map = {col.name: col.display_name for col in columns.value}
    all_items = result.value

    rows = []

    for item in all_items:
        if item.fields:
            # ids.append(item.fields.id)
            item.fields.additional_data['id'] = item.fields.id
            rows.append(item.fields.additional_data)

    df = pd.DataFrame(rows)
    df.rename(columns=column_map, inplace=True)
    return df


initiatives = await sp_list_to_df(site_id,list_ids[0],credential)
rps = await sp_list_to_df(site_id,list_ids[1],credential)
updates = await sp_list_to_df(site_id,list_ids[2],credential)

List Name: Book of Business - Initiatives, List ID: a09778e2-afc9-4996-a9b1-a61c313187c7
List Name: Book of Business - Reporting Periods, List ID: 57200f23-a3f0-4746-b794-76babd1a5f2a
List Name: Book of Business - Weekly Updates, List ID: f9edc97c-2e5c-4d4b-93e3-772d6bd450e5


In [ ]:
initiatives.rename(columns={'id':'initiative_id'},inplace=True)
rps = rps.loc[:, ~rps.columns.duplicated(keep='first')]
rps.rename(columns={'id':'reporting_period_id','Title':'reporting_period'},inplace=True)

In [ ]:
#create new rows df that includes cross between all initiative and reporting period ids 2/26/26
#will update so script creates df for only the new active reporting period 
new_rows = pd.merge(initiatives[['initiative_id']],rps[['reporting_period_id']],how='cross')
new_rows = pd.merge(new_rows,initiatives[['Initiative','initiative_id','Team','LeadLookupId']],on='initiative_id',how='left')
new_rows = pd.merge(new_rows,rps[['reporting_period','reporting_period_id','Is Active']],on='reporting_period_id',how='left')
new_rows['Title'] = new_rows['reporting_period'] + ' — ' +  new_rows['Initiative']

updates = updates.loc[:, ~updates.columns.duplicated(keep='first')]
new_rows = pd.merge(new_rows,updates[['Title','InitiativeLookupId','Status']],on='Title',how='left')
new_rows['Status'] = 'On track'

#Only including all the columns necessary for the new report rows in the active period to appear
new_rows = new_rows[['Title','Team','Initiative','Status','LeadLookupId','initiative_id','reporting_period_id','Is Active']]

#column names in dataframe must match the column names retrieved from Graph API, display/other names will be rejected
new_rows.rename(columns={'initiative_id':'InitiativeLookupId','reporting_period_id':'Reporting_x0020_PeriodLookupId','Is Active':'ActiveReportingPeriod'},inplace=True)

,Title,Team,Initiative,Status,LeadLookupId,InitiativeLookupId,Reporting_x0020_PeriodLookupId,ActiveReportingPeriod
0,Week ending 2026-01-23 — 311 Public Facing Portal,Data and Digital,311 Public Facing Portal,On track,11,1,1,No
1,Week ending 2026-01-16 — 311 Public Facing Portal,Data and Digital,311 Public Facing Portal,On track,11,1,2,No
2,Week ending 2026-02-27 — 311 Public Facing Portal,Data and Digital,311 Public Facing Portal,On track,11,1,3,Yes
3,Week ending 2026-01-23 — Accela Data and Dashb...,Data and Digital,Accela Data and Dashboard Support,On track,16,2,1,No
4,Week ending 2026-01-16 — Accela Data and Dashb...,Data and Digital,Accela Data and Dashboard Support,On track,16,2,2,No
...,...,...,...,...,...,...,...,...
133,Week ending 2026-01-16 — Fire & EMS Stat Support,Performance,Fire & EMS Stat Support,On track,37,46,2,No
134,Week ending 2026-02-27 — Fire & EMS Stat Support,Performance,Fire & EMS Stat Support,On track,37,46,3,Yes
135,Week ending 2026-01-23 — Police Stat Support,Performance,Police Stat Support,On track,37,47,1,No
136,Week ending 2026-01-16 — Police Stat Support,Performance,Police Stat Support,On track,37,47,2,No


In [ ]:
async def upload_new_items(site_id, list_id, df, credential):

    graph_client = GraphServiceClient(credential)

    async def upload_row(row):

        # remove metadata + NaN values
        payload = {
            col: (None if pd.isna(val) else val)
            for col, val in row.items()
            if col != "_item_id"   # skip existing IDs
        }

        #turns payload into object that can be ingested by Microsoft Graph API
        fields = FieldValueSet()
        fields.additional_data = payload

        item = ListItem()
        item.fields = fields

        await graph_client.sites.by_site_id(site_id).lists.by_list_id(list_id).items.post(item)
    #runs all the uploads concurrently saving time instead of uploading row 1 and then waiting for row 2
    await asyncio.gather(*(upload_row(row) for _, row in df.iterrows()))

await upload_new_items(site_id,list_ids[2],new_rows,credential)
